### Test ML Embeddings

In [1]:
from parse import get_data
import pandas as pd
import numpy as np

########################### EXTRACT DATA

def convert_labels(l) :
    if l == 'irr' :
        return [1, 0, 0, 0]
    elif l == 'neg' :
        return [0, 1, 0, 0]
    elif l == 'neu' :
        return [0, 0, 1, 0]
    else :
        return [0, 0, 0, 1]
    
def decode_output(out_array) :
    if out_array[0] == np.max(out_array) :
        return "irr", np.max(out_array)
    elif out_array[1] == np.max(out_array) :
        return "neg", np.max(out_array)
    elif out_array[2] == np.max(out_array) :
        return "neu",np.max(out_array)
    elif out_array[3] == np.max(out_array) :
        return "pos",np.max(out_array)
    else :
        return "err",0

# text, label = get_data("corpus_ready.data")
text, label = get_data("data/corpus.data")
labels = ["irr", "neg", "neu", "pos"]
label_data = np.array([convert_labels(l) for l in label])
print("size of train data : ", len(text))

######################### VECTORIZE DATA

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from functools import reduce

estimated_vocab_size = 2000
encoded = [one_hot(t, estimated_vocab_size) for t in text]
maxlength = np.max([len(x) for x in encoded])
encoded = pad_sequences(encoded, maxlen=maxlength, padding='post')

size of train data :  4998


Using TensorFlow backend.


In [3]:
percentage_train = 0.8
borne = int(percentage_train*len(text))
X_train = encoded[:borne]
Y_train = label_data[:borne]
X_test = encoded[borne:]
Y_test = label_data[borne:]
print("train set length : ",len(X_train), ", test set length : ",len(X_test))

train set length :  3998 , test set length :  1000


In [8]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding

# define the model
model = Sequential()
model.add(Embedding(input_dim=estimated_vocab_size, output_dim=8, input_length=maxlength))
model.add(Flatten())
model.add(Dense(4, activation='sigmoid'))

# compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# print(model.summary())
batch = 50
model.fit(X_train, Y_train, epochs=80, verbose=1)

Epoch 1/80
3998/3998 [==============================] - 0s 68us/step - loss: 0.5366 - acc: 0.7541
Epoch 2/80
3998/3998 [==============================] - 0s 29us/step - loss: 0.4520 - acc: 0.7915
Epoch 3/80
3998/3998 [==============================] - 0s 35us/step - loss: 0.4231 - acc: 0.8157
Epoch 4/80
3998/3998 [==============================] - 0s 33us/step - loss: 0.3924 - acc: 0.8334
Epoch 5/80
3998/3998 [==============================] - 0s 33us/step - loss: 0.3682 - acc: 0.8463
Epoch 6/80
3998/3998 [==============================] - 0s 32us/step - loss: 0.3475 - acc: 0.8585
Epoch 7/80
3998/3998 [==============================] - 0s 31us/step - loss: 0.3288 - acc: 0.8675
Epoch 8/80
3998/3998 [==============================] - 0s 32us/step - loss: 0.3105 - acc: 0.8742
Epoch 9/80
3998/3998 [==============================] - 0s 33us/step - loss: 0.2936 - acc: 0.8856
Epoch 10/80
3998/3998 [==============================] - 0s 35us/step - loss: 0.2769 - acc: 0.8902
Epoch 11/80
3998/39

In [9]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
# loss, accuracy = model.evaluate(X_train, Y_train)
print(accuracy)

0.7785


In [14]:
test = "Macron est un mauvais président bordel gilet jaunes"
test_oh = [one_hot(test, estimated_vocab_size)]
test_oh = pad_sequences(test_oh, maxlen=maxlength, padding='post')
print(type(test_oh))
print(test_oh)
out = list(model.predict(test_oh)[0])
decode_output(out)
# print("tweet is evaluated " + out[0] + " at " + str(out[1]) + "%")

<class 'numpy.ndarray'>
[[1274 1203 1134 1388 1519  974 1039 1652    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]


('neg', 0.23821595)